# AGENT1: l'agent qui simulait

Simulation du premier agent Sartrien qui une mémoire binaire: l'être ou le néant :-)

# Environnement

Si action `feel` alors l'environnement renvoie l'outcome correspondant à la position courate. 
Si action `moove`alors l'environnement passe à la position suivante et renvoie outcome 0

In [3]:
ACTION_SET = 0
ACTION_RESET = 1
ACTION_FEEL = 2
OUTCOME_EMPTY = 0
OUTCOME_FULL = 1

class Environment1:
    def __init__(self):
        self.state = 0
    def outcome(self, action):
        if action == ACTION_FEEL:
            outcome = self.state
        else:
            if action == ACTION_SET:
                self.state = 1
            else:
                self.state = 0
            outcome = 0
        return outcome

# L'agent

In [4]:
class Interaction:
    """An interaction is a tuple (action, outcome) with a valence"""
    def __init__(self, action, outcome, valence):
        self.action = action
        self.outcome = outcome
        self.valence = valence

    def key(self):
        """ The key to find this interaction in the dictinary is the string '<action><outcome>'. """
        return f"{self.action}{self.outcome}"

    def __str__(self):
        """ Print interaction in the form '<action><outcome:<valence>' for debug."""
        return f"{self.action}{self.outcome}:{self.valence}"

    def __eq__(self, other):
        """ Interactions are equal if they have the same key """
        return self.key() == other.key()

l'agent a un simulateur interne. 
Si action `set` il mets la mémoire à 1. 
Si action `reset` il mets la mémoire à 0. 
Si action `feel` il renvoie l'état de la mémoire.

In [13]:
import pandas as pd

class Agent:
    """Creating our agent"""
    def __init__(self, _interactions, simulator):
        """ Initialize the dictionary of interactions"""
        self._interactions = {interaction.key(): interaction for interaction in _interactions}
        self._intended_interaction = self._interactions["00"]
        self.memory = 0
        self.action_df = pd.DataFrame({"action": [i.action for i in _interactions if i.outcome == OUTCOME_EMPTY]})
        self.simulator = simulator

    def simulate(self, action):
        if action == ACTION_FEEL: 
            return self.memory
        else:
            if action == ACTION_SET:
                self.state = 1
            else:
                self.state = 0
            return 0

    def action(self, _outcome):
        """ Tracing the previous cycle """
        previous_interaction = self._interactions[f"{self._intended_interaction.action}{_outcome}"]
        print(f"Action: {self._intended_interaction.action}, Prediction: {self._intended_interaction.outcome}, Outcome: {_outcome}, " 
              f"Prediction: {self._intended_interaction.outcome == _outcome}, Valence: {previous_interaction.valence})")

        """ Computing the next interaction to try to enact """
        # Select the next action
        # Roll the actions
        self.action_df = pd.concat([self.action_df.tail(1), self.action_df.head(len(self.action_df) - 1)], ignore_index=True)
        # self.action_df = pd.concat([self.action_df.iloc[1:], self.action_df.iloc[[0]]], ignore_index=True)
        # Select the first action in the action_df
        intended_action = self.action_df.loc[0, "action"]
        
        # Predict the outcome based on simulation
        # intended_outcome = self.simulate(intended_action)
        intended_outcome = self.simulator.outcome(intended_action)
        # Memorize the intended interaction
        self._intended_interaction = self._interactions[f"{intended_action}{intended_outcome}"]
        return intended_action

# Run the simulation

In [14]:
interactions = [
    Interaction(ACTION_SET,OUTCOME_EMPTY,-1),
    Interaction(ACTION_SET,OUTCOME_FULL,1),
    Interaction(ACTION_RESET,OUTCOME_EMPTY,-1),
    Interaction(ACTION_RESET,OUTCOME_FULL,1),
    Interaction(ACTION_FEEL,OUTCOME_EMPTY,-1),
    Interaction(ACTION_FEEL,OUTCOME_FULL,1),
]

e = Environment1()
a = Agent(interactions, e)

outcome = 0
for i in range(10):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 2, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 1, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 2, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1)
Action: 1, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 2, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1)
Action: 1, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1)
